# Beat AML ML project part 2- Machine learning model to predict AC220 response

**SPOILER ALERT: This notebook made me realize that I could not build an accurate classifier, If you want to skip to the next notebook (Part 3) you will not miss anything.**

**Background:** This is the second notebook of the series. If you have not already done so please visit the first notebook for background information on the project and the general plan. In that first notebook we obtained a dataset which contain clinical information, genetic information (mutations), gene expression information and drug sensitivity information of over 400 Acute Myeloid Leukemia (AML) tumor biopsies. This information came from the recent [BEAT AML publication by Tyner et al in Nature](https://www.nature.com/articles/s41586-018-0623-z). We cleaned and combined the clinical, genetic and gene expression datasets into one data set, which will form our features. We are trying to predict whether or not patients will respond to a given therapy with this information. Then for those patients that do respond, we are trying to undertsnad the extent of their response. We can achieve this by stacking two models. First a classifier to catogorize patients into responders and non-responders, then a regressor to predict the extent of response. Tyner et al. tested 112 different therapies against these 400+ patients. This means we will have to solve 112 classifcation and regression problems. **Here I will focus on making a classifier/regressor, and interpreting that classifier, for just one treatment.** Specifically we will try to differentiate the patients that respond from those that do not respond at all.

**Information abour AC220:** I would like to justify why I chose AC220 as my example case in this notebook. AC220 is of personal interest to me, because I work with it in my day job as an AML researcher. AC220 is also known as Quizartinib. It is currently uner investigation in clinical trials to determine its efficacy either alone or in combination with chemotherapy. The reason why AC220 is so interesting is because it was specifically meant to treat one subset of AML patients, those that have the internal tandem duplication (ITD) mutation in a gene called Flt3. Without getting into too much detail, when Flt3 is mutated with an ITD it is always in an 'on' state which tells the cell to proliferate, divide and grow. Basically Flt3-ITD drive the cancer phenotype. AC220 is one treatment that is meant to turn Flt3-ITD off. The interesting thing is, from the currently published clinical data, we know that not all patients with Flt3-ITD respond to AC220. Also some patients that don't have Flt3-ITD also respond to AC220. It would be really interesting if we could figure out what features determine AC220 sensitivity. 

## Imports and datasets

Lets import our packages and load the dataframes from the previous notebook

In [10]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.size'] = 24
import seaborn as sns
sns.set(font_scale = 2)

# No warnings about setting value on copy of slice
pd.options.mode.chained_assignment = None

# Imputing missing values and scaling values
from sklearn.preprocessing import MinMaxScaler

#Import train test split
from sklearn.model_selection import train_test_split

# Import Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# Import Machine Learning models for Classification
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# Import for scoring 
from sklearn.metrics import accuracy_score, make_scorer, confusion_matrix, f1_score
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
# Reload the Target and Features dataframes

Features = pd.read_csv('Features.csv')
Features.set_index('LabId', inplace=True)

Targets = pd.read_csv('Targets.csv')
Targets.set_index('LabId', inplace=True)

Features.head()

,CEBPA_Biallelic,isRelapse,isDenovo,isTransformed,priorMalignancyNonMyeloid,cumulativeChemo,priorMalignancyRadiationTx,priorMDS,priorMDSMoreThanTwoMths,priorMDSMPN,...,GALNT7exp,C1RLexp,HLA-Fexp,EIF4A1exp,NARFexp,YPEL2exp,BTN3A2exp,TUBB6exp,MAT2Bexp,AP1S2exp
LabId,,,,,,,,,,,,,,,,,,,,,
13-00098,0,0,1,0,0,1,0,0,0,0,...,59.160975,33.539380,42.109566,225.642033,59.516830,52.488684,45.252956,12.514251,41.190273,49.849423
13-00118,0,1,0,0,0,1,0,0,0,0,...,135.493625,90.068184,39.047906,97.924147,61.485233,66.065462,100.272240,75.834686,101.344825,65.949507
13-00149,0,0,0,0,1,1,0,0,0,0,...,138.230096,31.928819,28.304911,56.609823,87.111042,38.051025,86.946318,0.576531,134.798366,61.084799
13-00157,0,0,1,0,0,1,0,0,0,0,...,148.073938,56.587364,45.798333,36.605639,53.229553,65.091982,77.092026,118.982088,137.229861,135.193157
13-00160,0,0,0,1,0,1,0,1,1,0,...,27.502231,62.321038,68.885714,58.648291,88.001355,43.725944,19.954300,39.185618,51.216037,60.585882


## Reshape the Target and Features datasets

Next, lets pull out only the AC220 data from our Targets dataset, convert the IC<sub>50</sub> values to a yes/no classification and merge it to the Features dataset. All non-responders were labeled with 10 as the IC<sub>50</sub> values. I will assume any label higher than 8 is a non reponder.

We can use our function from the last workbook to get the AC220 data from Targets. We can define a function to make our labels (1 for responders and 0 for non responders). We can then merge the labels column to the features dataset to make our final dataset.

In [12]:
# Define the function for pulling out AC220 data
def Get_Target (dataset, compound, measurement = 'ic50'):
    Target = dataset[dataset['inhibitor'].str.contains(compound)]
    Target = Target[measurement]
    return Target

# Define the function for making the labels (If doing classification)
def make_label (data, threshold):
    bins = [0, threshold, 10]
    labels = [0,1]    
    label = pd.cut(data, bins = bins, labels = labels)
    return label

# Run function to pull out AC220 IC50 values and make the labels (Classification)
Target = make_label(Get_Target(Targets, 'AC220'), 2)

# Keep only the rows of Target and Features that are in both dataframes (both)
idx = Target.index.intersection(Features.index)
Target = Target.loc[idx]
Features = Features.loc[idx]

# Find the shape of the new dataframes
print(Target.shape, Features.shape)

(247,) (247, 2687)


Okay now we have our target and features datasets. Unfortunately it looks like there was a lot less overlapp between the Taget an Features datasets. This is probably due to limited sample from each patient. Hopefully 247 samples is still enough that we can learn something!

Lets count how many responder we have in the dataset.

In [13]:
# Count the number of non-reponders
count = 0
for i in Target:
    if i == 1: count = count +1
    else: continue
print(count)

67


So we have 204 patients that are responders. This means our dataset is skewed to include mostly responder, even though we know that only ~50% of patients actually respond (see first notebook). 

## Split the data into training, test and validation datasets

Now lets split our data into training, test and validation data sets. We will use sklearn's Train_test_split here.

If you are not familiar, The training set is what we will use to optimize and train our model, while the test set is what we can use to test the model on untrained data. I opt to keep the test size small (20%) because we have such a small dataset, it would be a shame to lose more of it to testing. The validation set is untouched by the model, while optimizing the model for the test set, because our dataset is so small and it is unlikely that this model will be deployed in the real world, I decided to forgo the validation set.

The random_state parameter allows us to reproducibly get the same split (if we keep the parameter unchanged. The convention in the machine learning field is to name your features with a capital X and targets with a lowercase y, which is what I will do here.

In [14]:
# Split the remaining data into 80% training and 20% testing set
X, X_test, y, y_test = train_test_split(Features, Target, test_size = 0.2, random_state = 0)

# Find out the shape of the datasets
print(X.shape)
print(y.shape)
print(X_test.shape)
print(y_test.shape)

(197, 2687)
(197,)
(50, 2687)
(50,)


## Scaling the features

The final step to take before building the models is scaling the features. Some types of machine learning models do not require scaling (i.e. linear regression and random forest). In general I prefer to always scale the data, because we can always reverse the scaling (back transform) after we make our prediction if we have to.

Here we will apply a normalization to scale our data. For a given feature's normalization, the minimum value is subtracted from all data point and then all values are divided by the largest data point. **Using a standardization rather than a normalization could potentially improve the model.** This results in the largest value being set to 1 and the smallest value set to 0. We can easily do this using sklean MinMaxScaler.

In [15]:
# make a scaler object with a range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit to the training data
scaler.fit(X)

# Transform both the training and testing data with the scaler
X = scaler.transform(X)
X_test = scaler.transform(X_test)

#Then lets convert y to a one dimensional array so we can use it for machine learning
y = np.array(y).reshape((-1, ))
y_test = np.array(y_test).reshape((-1, ))

## Machine learning- Classification

Okay now we can try to make a classifier. Lets try out some of SKlearn's classifiers. I will try all of the following using mostly default options:
1. Logistic Regression
2. Multi-Layer Perceptron (Neural Net)
3. K-nearest neighbors
4. support vector classifier (Both linear and radial basis function)
5. Gaussian process classifier
6. Decision Tree
7. Random Forest Classifier
8. Ada boost Classifier
9. Gaussian Naive Bayes 
10. Quadratic Discrimination Analysis

We can then pick the best performing ones and work on hyper-parameter tuning those before we settle on the best performing classifier. The function below will return both the test and training scores. This will allow us to see accuracy, but also check for overfitting.

In [16]:
# Make a funtion to fit and score a model.
def fit_and_score(clf):
    clf.fit(X,y)
    return clf.score(X_test, y_test), clf.score(X,y)

# Logistic Regression
LR_clf, LR_clf2 = fit_and_score(LogisticRegression(C = 0.0001))

# MLPClassifier
MLP_clf, MLP_clf2 = fit_and_score(MLPClassifier(alpha=1))

# KNeighborsClassifier
KNN_clf, KNN_clf2 = fit_and_score(KNeighborsClassifier(3))

# SVC Linear
SVM_lin_clf, SVM_lin_clf2 = fit_and_score(SVC(kernel="linear", C=0.025))

# SVC rbf
SVM_clf, SVM_clf2 = fit_and_score(SVC(gamma=2, C=1))

# GaussianProcessClassifier
GausP_clf, GausP_clf2 = fit_and_score(GaussianProcessClassifier(1.0 * RBF(1.0)))

# DecisionTreeClassifier
DTC_clf, DTC_clf2 = fit_and_score(DecisionTreeClassifier(max_depth=5))

# RandomForestClassifier
RF_clf, RF_clf2 = fit_and_score(RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1))

# AdaBoostClassifier
AB_clf, AB_clf2 = fit_and_score(AdaBoostClassifier())

# GaussianNB
GNB_clf, GNB_clf2 = fit_and_score(GaussianNB())

# QuadraticDiscriminantAnalysis
Quad_clf, Quad_clf2 = fit_and_score(QuadraticDiscriminantAnalysis())

Classifiers = ['Logistic Regression', 'MLPClassifier', 'KNeighborsClassifier', 'SVC linear', 'SVC rbf', 
               'GaussianProcessClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 
               'GaussianNB', 'QuadraticDiscriminantAnalysis']
Test_accuracies = [LR_clf, MLP_clf, KNN_clf, SVM_lin_clf, SVM_clf, GausP_clf, DTC_clf, RF_clf, AB_clf, GNB_clf, Quad_clf]
Train_accuracies = [LR_clf2, MLP_clf2, KNN_clf2, SVM_lin_clf2, SVM_clf2, GausP_clf2, DTC_clf2, RF_clf2, AB_clf2, GNB_clf2, 
                   Quad_clf2]

for clf, train, test in zip(Classifiers, Train_accuracies, Test_accuracies):
    print('The {0} classifier has an training accuracy score of {1} and test score of {2}.'.format(clf, train, test))

The Logistic Regression classifier has an training accuracy score of 0.7157360406091371 and test score of 0.78.
The MLPClassifier classifier has an training accuracy score of 0.7157360406091371 and test score of 0.78.
The KNeighborsClassifier classifier has an training accuracy score of 0.8274111675126904 and test score of 0.7.
The SVC linear classifier has an training accuracy score of 0.9289340101522843 and test score of 0.78.
The SVC rbf classifier has an training accuracy score of 1.0 and test score of 0.78.
The GaussianProcessClassifier classifier has an training accuracy score of 1.0 and test score of 0.66.
The DecisionTreeClassifier classifier has an training accuracy score of 0.9543147208121827 and test score of 0.72.
The RandomForestClassifier classifier has an training accuracy score of 0.8426395939086294 and test score of 0.8.
The AdaBoostClassifier classifier has an training accuracy score of 1.0 and test score of 0.76.
The GaussianNB classifier has an training accuracy sco

C:\Program Files\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Program Files\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Let's focus on using the Logistic regression, Support vector, KNN and Random forest classifiers. These all have a score of at least 0.8 while not overfitting too much.


## Hyperparameter tuning

Here I will use GridSearchCV to find the best set of parameters. We will optimize for the best scoring parameters., we can then print the best parameters.

In [17]:
# Hyperparameter tuning for logistic regression
# Modified this code from https://chrisalbon.com/machine_learning/model_selection/hyperparameter_tuning_using_grid_search/

# Create logistic regression
logistic = LogisticRegression()

# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

# Function to maximize
_score = make_scorer(accuracy_score, greater_is_better=True)

# Create grid search using 5-fold cross validation
clf = GridSearchCV(logistic, hyperparameters, cv=5, verbose=0, scoring='accuracy')

# Fit grid search
best_model = clf.fit(X, y)

# View best hyperparameters and train/test score
print('Best Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Train score:', clf.score(X, y))
print('Test score:', clf.score(X_test, y_test))

Best Penalty: l2
Best C: 1.0
Train score: 1.0
Test score: 0.76


In [18]:
# Hyperparameter tuning for support vector classifier

# Create SVC
from sklearn.svm import SVC
SVC = SVC()

# Create kernal space
kernel = ['linear', 'poly', 'rbf', 'sigmoid']

# Create regularization hyperparameter space
C = np.logspace(-10, 4, 20)

gamma = np.logspace(-5, 4, 20)

# Create hyperparameter options
hyperparameters = dict(C=C, kernel=kernel, gamma=gamma)

# Function to maximize
#_score = make_scorer(accuracy_score, greater_is_better=True)

# Create grid search using 5-fold cross validation
clf = GridSearchCV(SVC, hyperparameters, cv=5, verbose=0, scoring = 'accuracy')

# Fit grid search
best_model = clf.fit(X, y)

# View best hyperparameters and train/test score
print('Best kernel:', best_model.best_estimator_.get_params()['kernel'])
print('Best C:', best_model.best_estimator_.get_params()['C'])
print('Best gamma:', best_model.best_estimator_.get_params()['gamma'])
print('Train score:', clf.score(X, y))
print('Test score:', clf.score(X_test, y_test))

Best kernel: poly
Best C: 0.012742749857031322
Best gamma: 0.02069138081114788
Train score: 0.949238578680203
Test score: 0.8


In [19]:
# Hyperparameter tuning for KNeighbors classifier

# Create KNN
KNN = KNeighborsClassifier()

# Create Number neighbors space
n_neighbors = [1,2,3,4,5,6,7,8,9,10]

# Create regularization hyperparameter space
weights = ['uniform', 'distance']
algorithm = ['ball_tree', 'kd_tree', 'brute']
leaf_size= [1,5,10,25,50,100]

# Create hyperparameter options
hyperparameters = dict(n_neighbors = n_neighbors, weights=weights, algorithm=algorithm, leaf_size= leaf_size)

# Function to maximize
#_score = make_scorer(accuracy_score, greater_is_better=True)

# Create grid search using 5-fold cross validation
clf = GridSearchCV(KNN, hyperparameters, cv=5, verbose=0, scoring ='accuracy')

# Fit grid search
best_model = clf.fit(X, y)

# View best hyperparameters and train/test score
print('Best n_neighbors:', best_model.best_estimator_.get_params()['n_neighbors'])
print('Best weights:', best_model.best_estimator_.get_params()['weights'])
print('Best algorithm:', best_model.best_estimator_.get_params()['algorithm'])
print('Best leaf_size:', best_model.best_estimator_.get_params()['leaf_size'])
print('Train score:', clf.score(X, y))
print('Test score:', clf.score(X_test, y_test))

Best n_neighbors: 1
Best weights: uniform
Best algorithm: ball_tree
Best leaf_size: 1
Train score: 1.0
Test score: 0.62


In [20]:
# Hyperparameter tuning for Random Forest Classifier

# Create Random forest classifier
RFC = RandomForestClassifier()

# Create max depth space
max_depth = np.linspace(1,10,10).astype(int)

# Create N-estimators space
n_estimators = np.linspace(5,500,10).astype(int)

# Create max features space
max_features = [1,2,4,8,12,24]

# Create hyperparameter options
hyperparameters = dict(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features)

# Function to maximize
#_score = make_scorer(accuracy_score, greater_is_better=True)

# Create grid search using 5-fold cross validation
clf = GridSearchCV(RFC, hyperparameters, cv=5, verbose=0, scoring ='accuracy')

# Fit grid search
best_model = clf.fit(X, y)

# View best hyperparameters and train/test score
print('Best max_depth:', best_model.best_estimator_.get_params()['max_depth'])
print('Best n_estimators:', best_model.best_estimator_.get_params()['n_estimators'])
print('Best max_features:', best_model.best_estimator_.get_params()['max_features'])
print('Train score:', clf.score(X, y))
print('Test score:', clf.score(X_test, y_test))

Best max_depth: 10
Best n_estimators: 60
Best max_features: 8
Train score: 1.0
Test score: 0.82


## Taking a closer look at the best classifiers

It seems that Gridsearch did not help improve our models too much. We only achieved modest improvements in each classifier. This limitation on model accuracy probably has to do with the fact that our dataset is small and that the number of responders is only ~20% of the dataset. Because our data is highly skewed to mostly include responders, it is possible that the model could just always classify as a responder and still have 80% accuracy. Lets take a look at the confusion matrix for each model and make sure they are accurately prediciting some of the non-responders in the test set.

In [21]:
#logistic
logistic = LogisticRegression(penalty='l2', C=1, solver='lbfgs')
logistic.fit(X,y)
print(logistic.score(X,y))
print(logistic.score(X_test,y_test))
print(confusion_matrix(y_test, logistic.predict(X_test)))
print(confusion_matrix(y, logistic.predict(X)))
print(f1_score(y_test, logistic.predict(X_test)))

1.0
0.76
[[32  7]
 [ 5  6]]
[[141   0]
 [  0  56]]
0.4999999999999999


In [22]:
#SVC
from sklearn.svm import SVC
SVC = SVC(kernel="poly", C=10000, gamma=0.00026366508987303583, degree=3)
SVC.fit(X,y)
print(SVC.score(X,y))
print(SVC.score(X_test,y_test))
print(confusion_matrix(y_test, SVC.predict(X_test)))
print(confusion_matrix(y, SVC.predict(X)))
print(f1_score(y_test, SVC.predict(X_test)))

0.9796954314720813
0.76
[[33  6]
 [ 6  5]]
[[141   0]
 [  4  52]]
0.45454545454545453


In [23]:
#KNeighbors    
KNN= KNeighborsClassifier(n_neighbors=6,weights='uniform',algorithm='ball_tree', leaf_size=1)
KNN.fit(X,y)
print(KNN.score(X,y))
print(KNN.score(X_test,y_test))
print(confusion_matrix(y_test, KNN.predict(X_test)))
print(confusion_matrix(y, KNN.predict(X)))
# print(f1_score(y_test, KNN.predict(X_test))) #F-1 metric doesn't really make sense for K-nearest neighbors classifier

0.7360406091370558
0.82
[[39  0]
 [ 9  2]]
[[139   2]
 [ 50   6]]


In [24]:
#Random Forest   
RFC = RandomForestClassifier(max_depth=3, n_estimators=5, max_features=8)
RFC.fit(X,y)
print(RFC.score(X,y))
print(RFC.score(X_test,y_test))
print(confusion_matrix(y_test, RFC.predict(X_test)))
print(confusion_matrix(y, RFC.predict(X)))
print(f1_score(y_test, RFC.predict(X_test)))

0.7817258883248731
0.82
[[39  0]
 [ 9  2]]
[[137   4]
 [ 39  17]]
0.3076923076923077


To be honest none of the classifier are performig that great! K-nearest neighbors and Random Forest classifiers are basically always classifying everything as responders (as I suspected some classifiers might be). Logistic regression and the support vector classifier are at least making an attempt to predict some non-responders in the test data, but neither are doing a great job. I decided to move on witht the support vector classifier since it is less over-fit and slightly more accurate on the test data (And therefore slightly higher F-1 score). 

Okay now lets save our best classifier and then we can focus on using regression to predict the extent of response.

In [25]:
Best_Classifier = SVC

## Interpreting the SVC model

The support vector classifier is a little bit harder to interpret than most of the classifiers sklearn has to offer. **Major caveat is that we know the classifier has some major flaws, so the interpretation can not be trusted anyway.** In fact the model can only give feature importances for the linear kernel as described in this [stack overflow post](https://stackoverflow.com/questions/41592661/determining-the-most-contributing-features-for-svm-classifier-in-sklearn).

Therefore, we can't interpret this model much.

## Conclusions

Unfortunately this attempt to make a classifier was not successful. I think this is mostly because of the small dataset size and the skewness towards responders. In the next notebook we will make a regression model that will help us determine the extent of response in those patients that do respond. 